# object detection pipeline (torchgeo + torchvision)

this notebook shows how to run training and inference with the scripts.


In [1]:

# type: ignore
!gdal_translate -co TILED=YES -co COMPRESS=NONE -co BIGTIFF=YES -co BLOCKXSIZE=512 -co BLOCKYSIZE=512 \
    ../data/Benton_Mosaic_Clip.tif ../data/Benton_Mosaic_Clip_prepared.tif


Input file size is 7312, 6937
0...10...20...30...40...50...60...70...80...90...100 - done.


In [3]:

# type: ignore

# !python ../scripts/train.py \
#     --task instance_seg \
#     --raster_path ../data/Benton_Mosaic_Clip_prepared.tif \
#     --normalize imagenet \
#     --labels_path ../data/BentonDetect_Clip.shp \
#     --out_dir ../outputs \
#     --tile_size 512 \
#     --epochs 12 \
#     --batch_size 2 \


!python ../scripts/inference.py \
    --task instance_seg \
    --raster_path ../data/Benton_Mosaic_Clip_prepared.tif \
    --normalize imagenet \
    --checkpoint ../outputs/model_final.pth \
    --out_vector ../outputs/detections.gpkg \
    --tile_size 1024 \
    --stride 512 \
    --score_thresh 0.5 \
    --nms_iou_thresh 0.5 \
    --device cuda

2026-02-02 15:59:22,864 INFO [debug] starting inference
2026-02-02 15:59:24,352 INFO [debug] model loaded and set to eval (device=cuda)
2026-02-02 15:59:24,379 INFO [debug] num_windows=182
2026-02-02 15:59:24,382 INFO [debug] processing window 1/182
2026-02-02 15:59:25,133 INFO [debug] processing window 2/182
2026-02-02 15:59:25,645 INFO [debug] processing window 3/182
2026-02-02 15:59:26,130 INFO [debug] processing window 4/182
2026-02-02 15:59:26,515 INFO [debug] processing window 5/182
2026-02-02 15:59:26,916 INFO [debug] processing window 6/182
2026-02-02 15:59:27,323 INFO [debug] processing window 7/182
2026-02-02 15:59:27,711 INFO [debug] processing window 8/182
2026-02-02 15:59:28,109 INFO [debug] processing window 9/182
2026-02-02 15:59:28,563 INFO [debug] processing window 10/182
2026-02-02 15:59:28,928 INFO [debug] processing window 11/182
2026-02-02 15:59:29,323 INFO [debug] processing window 12/182
2026-02-02 15:59:29,764 INFO [debug] processing window 13/182
2026-02-02 15: